### Introduction to Spark

I think you do hear `big data` these years, `big data` is just like traditional technologies but just data is too big that we couldn't use common tools to process it, like if you have 10,000 records data, you could just store it in MYSQL,  but if you have 100000000 records data, if have just store them in MYSQL, it couldn't handle it in fact, that's where big data comes in, there are many tools that we could use for `big data`,  but most common tools are `Spark` to process data, `HIVE` to store relational data, `HDFS` for storing distributed files(What means `distributed`, for big data, we couldn't just use one machine,but many machines, commonly one machine as `master` machine, the others are `workers` machines), so for these `big data` technologies are both distributed.

As there are many tools that we could use, but most popular tool is [`Spark`](https://spark.apache.org/), it's a unified analytics engine for large-scale data processing, it combines many tools like `SQL`, `Machine learning`, `streaming`, this notebook is try to show you how to use `Spark` to do machine learning also with `SQL`.

Let's get start.

In [71]:
# before do anything, maybe you should first install `pyspark` module for python
# just use pip command:   pip install pyspark

# if you installed the pyspark, but you couldn't import it, that's means you should install java also,
# if you don't know how to do that, I will show you with your computer
import numpy as np
import pandas as pd
import os
from sklearn.datasets import load_iris
import warnings
import tempfile
import shutil

# let's try to import pyspark module, 
import pyspark

# then let's try to import SparkSession, as spark is a distributed logic, we do need a entry for 
# code to start to run, that's SparkSession, if you are not familiar with it, never mind, just keep it in mind
from pyspark.sql import SparkSession
warnings.simplefilter('ignore')



In [72]:
# let's first to check what pyspark version we use
# so currently we use 2.3.2 version.
print("Spark version:", pyspark.__version__)

Spark version: 2.3.2


### Make data into a file

first let's try to store the iris data into a file, then we could use spark to read it, one thing to notice is that we use spark most time is for big data means that maybe the data is stored in distributed file system like `HDFS`, don't worry about `HDFS`, that's easy to use and understand, you could just take it as a system to store files(files means even for MYSQL, data is stored as a file or many files), but currently I just to make it into a local server, if in HDFS could just change one line code.

In [73]:
# first make a temp folder to store data file
tmp_path = tempfile.mkdtemp()

# currently I just to make the data into a file, in real project, they are just stored in somewhere.
x, y = load_iris(return_X_y=True)
data = np.concatenate([x, y.reshape(-1, 1)], axis=1)

    df = pd.DataFrame(data, columns=['a', 'b', 'c', 'd', 'label'])

# let's store the dataframe into a csv file
df.to_csv(os.path.join(tmp_path, 'data.csv'), index=False)   # we don't need index

# let's check, you could see that we have stored data file into tmp folder.
print("What is stored in path: {}, files: {}".format(tmp_path, os.listdir(tmp_path)))

What is stored in path: C:\Users\GUANGQ~1.LU\AppData\Local\Temp\tmp_8puox6t, files: ['data.csv']


### Load data(ETL step)

let's use spark to read it from local server, one thing to notice is that Spark also use `DataFrame`, that's it just like `pandas`'s dataframe, but spark's returned `DataFrame` is a distributed object. If you are not famaliar about `distributed` dataframe, I could just explain it with you less than 5 mins.

In [74]:
# before we do anything, we should first init `SparkSession`, so that we could interact with `distributed` system
# it maybe takes some time
spark = SparkSession.builder.getOrCreate()     # most times should just like this command

In [75]:
# after we have get sparksesion object, then we could use it to read data
# these two options:`.option('header', True).option('inferSchema', True)` is to keep the csv header for new dataframe.

df_spark = spark.read.format('csv').option('header', True).option('inferSchema', True).load(os.path.join(tmp_path, 'data.csv'))

In [76]:
# let's check it, what it is: so you could see that it's `pyspark.sql.dataframe.DataFrame`, that's a DataFrame
type(df_spark)

pyspark.sql.dataframe.DataFrame

In [77]:
# let's check what is contains
# most time is `show` function, it will show 20 rows by default
df_spark.show()

+---+---+---+---+-----+
|  a|  b|  c|  d|label|
+---+---+---+---+-----+
|5.1|3.5|1.4|0.2|  0.0|
|4.9|3.0|1.4|0.2|  0.0|
|4.7|3.2|1.3|0.2|  0.0|
|4.6|3.1|1.5|0.2|  0.0|
|5.0|3.6|1.4|0.2|  0.0|
|5.4|3.9|1.7|0.4|  0.0|
|4.6|3.4|1.4|0.3|  0.0|
|5.0|3.4|1.5|0.2|  0.0|
|4.4|2.9|1.4|0.2|  0.0|
|4.9|3.1|1.5|0.1|  0.0|
|5.4|3.7|1.5|0.2|  0.0|
|4.8|3.4|1.6|0.2|  0.0|
|4.8|3.0|1.4|0.1|  0.0|
|4.3|3.0|1.1|0.1|  0.0|
|5.8|4.0|1.2|0.2|  0.0|
|5.7|4.4|1.5|0.4|  0.0|
|5.4|3.9|1.3|0.4|  0.0|
|5.1|3.5|1.4|0.3|  0.0|
|5.7|3.8|1.7|0.3|  0.0|
|5.1|3.8|1.5|0.3|  0.0|
+---+---+---+---+-----+
only showing top 20 rows



### ETL follow 

After we have already get dataframe, then we could do other transformations aka `ETL`, what we could do with pandas dataframe, we could do it by spark.

Let's show you.

In [78]:
# let's check spark dataframe columns type
df_spark.printSchema()

root
 |-- a: double (nullable = true)
 |-- b: double (nullable = true)
 |-- c: double (nullable = true)
 |-- d: double (nullable = true)
 |-- label: double (nullable = true)



One thing to notice is that if we want to use `machine learning`, most functions assume each column is a `Vector` type, currently we just get `double` type. 

Let's convert `double` type into `vector` type.

In [79]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=['a'], outputCol='a_vector')

# we don't need to fit the `VectorAssembler`, we could just use it to transform
df_spark_vector = assembler.transform(df_spark)

# let's check the new dataframe, so we could see that we do add a new column
# called `a_vector`, right?
df_spark_vector.show(5)

+---+---+---+---+-----+--------+
|  a|  b|  c|  d|label|a_vector|
+---+---+---+---+-----+--------+
|5.1|3.5|1.4|0.2|  0.0|   [5.1]|
|4.9|3.0|1.4|0.2|  0.0|   [4.9]|
|4.7|3.2|1.3|0.2|  0.0|   [4.7]|
|4.6|3.1|1.5|0.2|  0.0|   [4.6]|
|5.0|3.6|1.4|0.2|  0.0|   [5.0]|
+---+---+---+---+-----+--------+
only showing top 5 rows



In [80]:
# maybe we want to standard the `a` column, we could use spark functions to do it
# for standard belongs to `machine learning`, so it is in `ml` package.
from pyspark.ml.feature import StandardScaler

# we have to define which column to process and define the output column.
scaler = StandardScaler(inputCol='a_vector', outputCol='a_standard', withMean=True, withStd=True)

# this is to fit model, returned `scaler_model` is the trained model based on the new dataframe: `df_spark_vector`
scaler_model = scaler.fit(df_spark_vector)

# let's use the trained model to get transform data
df_scaler = scaler_model.transform(df_spark_vector)

# let's check the new dataframe, we want to see whole columns, we could set truncate to false,
# then we could see whole things in columns.
df_scaler.show(5, truncate=False)


+---+---+---+---+-----+--------+---------------------+
|a  |b  |c  |d  |label|a_vector|a_standard           |
+---+---+---+---+-----+--------+---------------------+
|5.1|3.5|1.4|0.2|0.0  |[5.1]   |[-0.8976738791967643]|
|4.9|3.0|1.4|0.2|0.0  |[4.9]   |[-1.1392004834649512]|
|4.7|3.2|1.3|0.2|0.0  |[4.7]   |[-1.3807270877331392]|
|4.6|3.1|1.5|0.2|0.0  |[4.6]   |[-1.5014903898672336]|
|5.0|3.6|1.4|0.2|0.0  |[5.0]   |[-1.0184371813308577]|
+---+---+---+---+-----+--------+---------------------+
only showing top 5 rows



### Advanced function

Sometimes, we would like to use our logic to process data as we want, then we would need to use spark `udf` function(udf means user defined function), but if we learn deeper, then I could tell you how to use it in real project, but for now you could just see that we could do that with spark.

In [83]:
# before we go on, we could also use ourself defined function to do transformations
# let's show you. but we need to use udf logic in spark
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

# this is the main logic that we want to process for each row.
def convert_bin(data):
    if data >= 1.5:
        return 1.0
    else: 
        return 0.0

# this is to register a `udf` function.
convert_udf = udf(convert_bin, DoubleType())

# we could use our defined function to do the transformation with lambda function
df_scaler_udf = df_scaler.withColumn('c_bins', convert_udf(df_scaler["c"]))

# let's check the result
df_scaler_udf.show(5, truncate=False)

+---+---+---+---+-----+--------+---------------------+------+
|a  |b  |c  |d  |label|a_vector|a_standard           |c_bins|
+---+---+---+---+-----+--------+---------------------+------+
|5.1|3.5|1.4|0.2|0.0  |[5.1]   |[-0.8976738791967643]|0.0   |
|4.9|3.0|1.4|0.2|0.0  |[4.9]   |[-1.1392004834649512]|0.0   |
|4.7|3.2|1.3|0.2|0.0  |[4.7]   |[-1.3807270877331392]|0.0   |
|4.6|3.1|1.5|0.2|0.0  |[4.6]   |[-1.5014903898672336]|1.0   |
|5.0|3.6|1.4|0.2|0.0  |[5.0]   |[-1.0184371813308577]|0.0   |
+---+---+---+---+-----+--------+---------------------+------+
only showing top 5 rows



### Machine learning with Spark

Alright, we have seen that we could use spark to do `ETL`, most times, I do use `Spark` many times to do `ETL` and `machine learning`, I will show you how to do machine learning training with `Spark`, stay tuned, let's start.

In [84]:
# before we do training, we need to make spark dataframe columns into a vector type for training, just like `a_vector` logic

# most times, I would make the whole features that we need into one column called: `features`, you should do that either.
assembler_ml = VectorAssembler(inputCols=['a', 'b', 'c', 'd', 'a_standard', 'c_bins'], outputCol='features')

df_train = assembler_ml.transform(df_scaler_udf)

# let's last check with the trained dataframe
# we would just use the last column: `features` and 'label' for training. 
df_train.show(5, truncate=False)

+---+---+---+---+-----+--------+---------------------+------+-----------------------------------------+
|a  |b  |c  |d  |label|a_vector|a_standard           |c_bins|features                                 |
+---+---+---+---+-----+--------+---------------------+------+-----------------------------------------+
|5.1|3.5|1.4|0.2|0.0  |[5.1]   |[-0.8976738791967643]|0.0   |[5.1,3.5,1.4,0.2,-0.8976738791967643,0.0]|
|4.9|3.0|1.4|0.2|0.0  |[4.9]   |[-1.1392004834649512]|0.0   |[4.9,3.0,1.4,0.2,-1.1392004834649512,0.0]|
|4.7|3.2|1.3|0.2|0.0  |[4.7]   |[-1.3807270877331392]|0.0   |[4.7,3.2,1.3,0.2,-1.3807270877331392,0.0]|
|4.6|3.1|1.5|0.2|0.0  |[4.6]   |[-1.5014903898672336]|1.0   |[4.6,3.1,1.5,0.2,-1.5014903898672336,1.0]|
|5.0|3.6|1.4|0.2|0.0  |[5.0]   |[-1.0184371813308577]|0.0   |[5.0,3.6,1.4,0.2,-1.0184371813308577,0.0]|
+---+---+---+---+-----+--------+---------------------+------+-----------------------------------------+
only showing top 5 rows



In [85]:
# before we do training, we need to split data into train and test data
# we could first just to select the columns that we need: `features` and `label`
df_train_selected = df_train.select(['features', 'label'])

# then we could split it into train and test data with 8:2
(train_data, test_data) = df_train_selected.randomSplit([0.8, 0.2])

print("How many training: ", train_data.count())
print("How many testing: ", test_data.count())

How many training:  125
How many testing:  25


In [86]:
# let's start with machine learning with Logistic Regression in Spark
from pyspark.ml.classification import LogisticRegression

# when to init the object, we could set both `l1` and `l2` regularization.
lr = LogisticRegression(maxIter=50, regParam=0.3, elasticNetParam=0.8)

# let's start our training with training data
lr_model = lr.fit(train_data)

print("Model traning finished.")

# let's get the prediction on test data: for spark framework, the prediction function is `transform`
pred = lr_model.transform(test_data)

# let's check what's like for test data
# for probability is the probability of each class, you could just focus on `probability` and `prediction` column.
pred.show(5)

Model traning finished.
+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[4.4,3.0,1.3,0.2,...|  0.0|[0.51850706598155...|[0.53994585442645...|       0.0|
|[4.6,3.1,1.5,0.2,...|  0.0|[0.48105142290053...|[0.53062885971303...|       0.0|
|[4.7,3.2,1.3,0.2,...|  0.0|[0.52140163258939...|[0.54066479367159...|       0.0|
|[4.9,3.6,1.4,0.1,...|  0.0|[0.53760753814345...|[0.54830337835601...|       0.0|
|[5.0,3.3,1.4,0.2,...|  0.0|[0.50339745270084...|[0.53619036584358...|       0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 5 rows



In [87]:
pred.printSchema()

root
 |-- features: vector (nullable = true)
 |-- label: double (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



In [88]:
# after we have get prediction result, let's check the accuracy of our model with spark
# for spark, we need to convert dataframe into RDD, so that we could directly eveluate prediction
from pyspark.mllib.evaluation import MulticlassMetrics

# just get the `label` and `prediction` columns.
pred_evaluate = pred.select(['label', 'prediction']).rdd

metrics = MulticlassMetrics(pred_evaluate)

In [89]:
print("model accuracy on test data: {:.4f} %".format(metrics.accuracy * 100))

model accuracy on test data: 84.0000 %


In [90]:
# in fact, we could also get some other metrics of the prediction result based on
# the metrics object, like recall, precision, f2-score etc.

recall_score = metrics.recall()
precision_score = metrics.precision()
f1_score = metrics.fMeasure()
print("Model recall score:", recall_score)
print("Model precision score:", precision_score)
print("Model f1 score:", f1_score)

Model recall score: 0.84
Model precision score: 0.84
Model f1 score: 0.84


In [91]:
unique_label = df_train_selected.rdd.map(lambda x: x.label).distinct().collect()
unique_label

[0.0, 1.0, 2.0]

### Last words

Currently I just walk through of `Spark` that if you are not familiar with `big data` technologies, I have to say that for `Spark` is really good to use in real project. One thing to notice is that I just try to let you know what is `Spark` and how to use it, there are many other ways that we could use `Spark`, but if we do face the problem that could use `Spark` to process data like `ETL` or `machine learning`, I will try to guide you.


Hope you have a good understanding about `Spark` for `Big data`! 